In [1]:
%matplotlib inline
import findspark
import os

In [2]:
findspark.init(os.environ['SPARK_HOME'])

In [3]:
%matplotlib inline
import pyspark 
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
#from pyspark.ml.linalg import *
from pyspark.sql.functions import *  
#from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import OneHotEncoder, StringIndexer, SQLTransformer, Normalizer, VectorAssembler, VectorIndexer
from pyspark.ml import Pipeline
#from pyspark.ml.feature import SQLTransformer
from pyspark.ml.linalg import Vectors
#from pyspark.ml.feature import Normalizer
#from pyspark.ml.linalg import Vectors
#from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor,LinearRegression
#from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# creamos la sesión de spark
spark = SparkSession\
.builder\
.appName("spark_Parte2")\
.getOrCreate()

In [5]:
spark

In [6]:
profecoSchema = StructType([StructField("producto", StringType(), True), \
                     StructField("presentacion", StringType(), True), \
                     StructField("marca", StringType(), True), \
                     StructField("categoria", StringType(), True), \
                     StructField("catalogo", StringType(), True), \
                     StructField("precio", DoubleType(), True), \
                     StructField("fecharegistro", TimestampType(), True), \
                     StructField("cadenacomercial", StringType(), True), \
                     StructField("giro", StringType(), True), \
                     StructField("nombrecomercial", StringType(), True), \
                     StructField("direccion", StringType(), True), \
                     StructField("estado", StringType(), True), \
                     StructField("municipio", StringType(), True), \
                     StructField("latitud", DoubleType(), True), \
                     StructField("longitud", DoubleType(), True)] )

In [6]:
profecoDf = spark.read.format("csv")\
        .option("delimiter", "|")\
        .option("header","true")\
        .schema(profecoSchema) \
        .option("inferSchema", "true")\
        .load("profeco_final_bash.csv")

In [7]:
profecoDF = spark.read.format('parquet')\
        .load("profecoFinal.parquet")

In [8]:
profecoDF = profecoDF.repartition(400)

In [9]:
profecoDf.printSchema()

NameError: name 'profecoDf' is not defined

In [10]:
profecoDF.printSchema()

root
 |-- producto: string (nullable = true)
 |-- presentacion: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- catalogo: string (nullable = true)
 |-- precio: double (nullable = true)
 |-- fecharegistro: timestamp (nullable = true)
 |-- cadenacomercial: string (nullable = true)
 |-- giro: string (nullable = true)
 |-- nombrecomercial: string (nullable = true)
 |-- direccion: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- latitud: double (nullable = true)
 |-- longitud: double (nullable = true)



In [11]:
# Manipulación de variables
sqlTrans = SQLTransformer( \
    statement="SELECT * FROM  __THIS__ \
          WHERE categoria LIKE '%medicamentos%' \
          AND estado='distrito federal' \
          AND MONTH(fecharegistro)=4 \
          AND YEAR(fecharegistro)=2016")

In [12]:
# Selección de variables
formula = SQLTransformer( \
#    statement="SELECT producto, marca, precio AS label ,cadenacomercial, municipio FROM  __THIS__ GROUP BY producto")
    statement="SELECT producto,marca,cadenacomercial, municipio, MEAN(precio) AS label FROM  __THIS__ GROUP BY producto,marca,cadenacomercial,municipio")

In [13]:
# Formato de String a categótico
productoIndexer = StringIndexer(inputCol="producto", outputCol="productoIndex")
marcaIndexer = StringIndexer(inputCol="marca", outputCol="marcaIndex")
cadenacomercialIndexer = StringIndexer(inputCol="cadenacomercial", outputCol="cadenacomercialIndex")
municipioIndexer = StringIndexer(inputCol="municipio", outputCol="municipioIndex")

In [14]:
assembler = VectorAssembler(
#    inputCols=["productoIndex", "marcaIndex","cadenacomercialIndex","municipioIndex"],
    inputCols=["productoIndex", "marcaIndex","cadenacomercialIndex","municipioIndex"],
    outputCol="Features")

In [15]:
# Train a random forest model
rf = RandomForestRegressor(labelCol='label',featuresCol='Features',maxBins=400)
pipeline = Pipeline(stages=[sqlTrans,formula,productoIndexer,marcaIndexer,cadenacomercialIndexer,municipioIndexer,assembler,rf])

In [16]:
paramGrid = ParamGridBuilder()\
    .addGrid(rf.maxDepth, [2,5,7])\
    .addGrid(rf.numTrees, [64,96,128])\
    .build()

In [17]:
tvs = TrainValidationSplit(estimator=pipeline,
                           estimatorParamMaps=paramGrid,
                           evaluator=RegressionEvaluator(),
                           # 80% of the data will be used for training, 20% for validation.
                           trainRatio=0.8)

In [18]:
model = tvs.fit(profecoDF)

In [19]:
model.estimator

Param(parent='TrainValidationSplitModel_fad83aff1cf4', name='estimator', doc='estimator to be cross-validated')

In [20]:
model.uid

'TrainValidationSplitModel_fad83aff1cf4'

In [21]:
model.bestModel

PipelineModel_a306ee1e9b04

In [22]:
model.explainParams()

"estimator: estimator to be cross-validated (current: Pipeline_324d9862c7e8)\nestimatorParamMaps: estimator param maps (current: [{Param(parent='RandomForestRegressor_3b1ecac09ed7', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 2, Param(parent='RandomForestRegressor_3b1ecac09ed7', name='numTrees', doc='Number of trees to train (>= 1).'): 64}, {Param(parent='RandomForestRegressor_3b1ecac09ed7', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 2, Param(parent='RandomForestRegressor_3b1ecac09ed7', name='numTrees', doc='Number of trees to train (>= 1).'): 96}, {Param(parent='RandomForestRegressor_3b1ecac09ed7', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 2, Param(parent='RandomForestRegressor_3b1ecac09ed7', name='numTre

In [23]:
model.extractParamMap()

{Param(parent='TrainValidationSplitModel_fad83aff1cf4', name='seed', doc='random seed.'): 2357993160371581517,
 Param(parent='TrainValidationSplitModel_fad83aff1cf4', name='estimator', doc='estimator to be cross-validated'): Pipeline_324d9862c7e8,
 Param(parent='TrainValidationSplitModel_fad83aff1cf4', name='estimatorParamMaps', doc='estimator param maps'): [{Param(parent='RandomForestRegressor_3b1ecac09ed7', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 2,
   Param(parent='RandomForestRegressor_3b1ecac09ed7', name='numTrees', doc='Number of trees to train (>= 1).'): 64},
  {Param(parent='RandomForestRegressor_3b1ecac09ed7', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 2,
   Param(parent='RandomForestRegressor_3b1ecac09ed7', name='numTrees', doc='Number of trees to train (>= 1).'): 96},
  {Param(parent='

In [24]:
model.bestModel

PipelineModel_a306ee1e9b04

In [25]:
model.getEstimatorParamMaps()

[{Param(parent='RandomForestRegressor_3b1ecac09ed7', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 2,
  Param(parent='RandomForestRegressor_3b1ecac09ed7', name='numTrees', doc='Number of trees to train (>= 1).'): 64},
 {Param(parent='RandomForestRegressor_3b1ecac09ed7', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 2,
  Param(parent='RandomForestRegressor_3b1ecac09ed7', name='numTrees', doc='Number of trees to train (>= 1).'): 96},
 {Param(parent='RandomForestRegressor_3b1ecac09ed7', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 2,
  Param(parent='RandomForestRegressor_3b1ecac09ed7', name='numTrees', doc='Number of trees to train (>= 1).'): 128},
 {Param(parent='RandomForestRegressor_3b1ecac09ed7', name='maxDepth',

In [26]:
model.validationMetrics

[217.71386505318395,
 223.64828110837172,
 218.77543591397645,
 128.41655622525496,
 130.76820536261008,
 127.2947353609827,
 110.25212820520983,
 108.98707655254567,
 108.32031640575397]